In [67]:
import pandas as pd
import numpy as np
import matplotlib

In [128]:
df_gasoline_price = pd.read_csv('data/U.S._All_Grades_Conventional_Retail_Gasoline_Prices_Weekly.csv', skiprows = 4)

In [129]:
df_gasoline_price.head(5)
df_gasoline_price.dtypes

Week of                                                       object
Series ID: PET.EMM_EPM0U_PTE_NUS_DPG.W Dollars per Gallon    float64
dtype: object

In [130]:
df_gasoline_price.rename(columns={'Week of': 'Gas_Price_Date', 'Series ID: PET.EMM_EPM0U_PTE_NUS_DPG.W Dollars per Gallon': 'Gas_price_per_gallon'}, inplace=True)

In [131]:
df_gasoline_price['Gas_Price_Date'] = pd.DatetimeIndex(df_gasoline_price['Gas_Price_Date'])
df_gasoline_price['Date'] = df_gasoline_price['Gas_Price_Date'] -  pd.to_timedelta(3, unit='d')

In [123]:
df_gasoline_price['Date'] = df_gasoline_price['Date'].dt.strftime('%Y-%m-%d')

AttributeError: Can only use .dt accessor with datetimelike values

In [132]:
df_gasoline_price['Date'] = df_gasoline_price['Date'].dt.strftime('%m/%d/%Y')

In [134]:
df_gasoline_price['Gas_Price_Date'] = df_gasoline_price['Gas_Price_Date'].dt.strftime('%m/%d/%Y')

In [135]:
df_gasoline_price.head(5)

,Gas_Price_Date,Gas_price_per_gallon,Date
0,04/04/2022,4.096,04/01/2022
1,03/28/2022,4.152,03/25/2022
2,03/21/2022,4.165,03/18/2022
3,03/14/2022,4.252,03/11/2022
4,03/07/2022,4.031,03/04/2022


In [72]:
df_crude_spot_price = pd.read_csv('data/Cushing_OK_WTI_Spot_Price_FOB_Weekly.csv', skiprows = 4)

In [73]:
#Daily price
df_crude_spot_price.head()

,Week of,Series ID: PET.RWTC.W Dollars per Barrel
0,03/25/2022,113.69
1,03/18/2022,100.43
2,03/11/2022,113.39
3,03/4/2022,106.80
4,02/25/2022,92.18


In [74]:
df_crude_spot_price.rename(columns={'Week of': 'Date', 'Series ID: PET.RWTC.W Dollars per Barrel': 'crude_price_per_barrel'}, inplace=True)

In [75]:
df_crude_spot_price.head()

,Date,crude_price_per_barrel
0,03/25/2022,113.69
1,03/18/2022,100.43
2,03/11/2022,113.39
3,03/4/2022,106.80
4,02/25/2022,92.18


In [76]:
df_stock_gasoline = pd.read_csv('data/U.S._Ending_Stocks_of_Finished_Motor_Gasoline_Weekly.csv', skiprows = 4)

In [77]:
df_stock_gasoline.head()

,Week of,Series ID: PET.WGFSTUS1.W Thousand Barrels
0,03/25/2022,18971
1,03/18/2022,18344
2,03/11/2022,17123
3,03/4/2022,19062
4,02/25/2022,17714


In [78]:
df_stock_gasoline.rename(columns={'Week of': 'Date', 'Series ID: PET.WGFSTUS1.W Thousand Barrels': 'Stocks_per_thousand_barrels'}, inplace=True)

In [79]:
df_stock_gasoline.head()

,Date,Stocks_per_thousand_barrels
0,03/25/2022,18971
1,03/18/2022,18344
2,03/11/2022,17123
3,03/4/2022,19062
4,02/25/2022,17714


In [80]:
df_gasoline_supply = pd.read_csv('data/U.S._Days_of_Supply_of_Total_Gasoline_Weekly.csv', skiprows = 4)

In [81]:
df_gasoline_supply.head()

,Week of,Series ID: PET.W_EPM0_VSD_NUS_DAYS.W Number of Days
0,03/25/2022,27.3
1,03/18/2022,27.0
2,03/11/2022,27.3
3,03/4/2022,28.0
4,02/25/2022,28.0


In [82]:
df_gasoline_supply.rename(columns={'Week of': 'Date', 'Series ID: PET.W_EPM0_VSD_NUS_DAYS.W Number of Days': 'No_of_days'}, inplace=True)

In [83]:
df_gasoline_supply.head()

,Date,No_of_days
0,03/25/2022,27.3
1,03/18/2022,27.0
2,03/11/2022,27.3
3,03/4/2022,28.0
4,02/25/2022,28.0


In [84]:
df_refinery_capacity = pd.read_csv('data/U.S._Percent_Utilization_of_Refinery_Operable_Capacity_Weekly.csv', skiprows = 4)

In [85]:
df_refinery_capacity.head()

,Week of,Series ID: PET.WPULEUS3.W Percent
0,03/25/2022,92.1
1,03/18/2022,91.1
2,03/11/2022,90.4
3,03/4/2022,89.3
4,02/25/2022,87.7


In [86]:
df_refinery_capacity.rename(columns={'Week of': 'Date', 'Series ID: PET.WPULEUS3.W Percent': 'Utilization_Percentage'}, inplace=True)

In [87]:
df_refinery_capacity.head()

,Date,Utilization_Percentage
0,03/25/2022,92.1
1,03/18/2022,91.1
2,03/11/2022,90.4
3,03/4/2022,89.3
4,02/25/2022,87.7


In [88]:
df_crude_exports = pd.read_csv('data/U.S._Exports_of_Crude_Oil_Weekly.csv', skiprows = 4)

In [89]:
df_crude_exports.head()

,Week of,Series ID: PET.WCREXUS2.W Thousand Barrels per Day
0,03/25/2022,2988
1,03/18/2022,3844
2,03/11/2022,2936
3,03/4/2022,2422
4,02/25/2022,3796


In [90]:
df_crude_exports.rename(columns={'Week of': 'Date', 'Series ID: PET.WCREXUS2.W Thousand Barrels per Day': 'Exp_Thousand_Barrels_Per_Day'}, inplace=True)

In [91]:
df_crude_exports.head()

,Date,Exp_Thousand_Barrels_Per_Day
0,03/25/2022,2988
1,03/18/2022,3844
2,03/11/2022,2936
3,03/4/2022,2422
4,02/25/2022,3796


In [92]:
df_crude_imports = pd.read_csv('data/U.S._Imports_of_Crude_Oil_Weekly.csv', skiprows = 4)

In [93]:
df_crude_imports.head()

,Week of,Series ID: PET.WCRIMUS2.W Thousand Barrels per Day
0,03/25/2022,6259
1,03/18/2022,6486
2,03/11/2022,6395
3,03/4/2022,6319
4,02/25/2022,5767


In [94]:
df_crude_imports.rename(columns={'Week of': 'Date', 'Series ID: PET.WCRIMUS2.W Thousand Barrels per Day': 'Imp_Thousand_Barrels_Per_Day'}, inplace=True)

In [95]:
df_crude_imports.head()

,Date,Imp_Thousand_Barrels_Per_Day
0,03/25/2022,6259
1,03/18/2022,6486
2,03/11/2022,6395
3,03/4/2022,6319
4,02/25/2022,5767


In [96]:
#df_temp_1 = pd.merge(df_gasoline_price, df_crude_spot_price, on = ['Date'], how = 'inner')
df_temp_2 = pd.merge(df_crude_spot_price, df_stock_gasoline, on = ['Date'], how = 'inner')
df_temp_3 = pd.merge(df_temp_2, df_gasoline_supply, on = ['Date'], how = 'inner')
df_temp_4 = pd.merge(df_temp_3, df_refinery_capacity, on = ['Date'], how = 'inner')
df_temp_5 = pd.merge(df_temp_4, df_crude_exports, on = ['Date'], how = 'inner')
df_merged = pd.merge(df_temp_5, df_crude_imports, on = ['Date'], how = 'inner')

In [126]:
df_temp_1 = pd.merge(df_crude_spot_price, df_stock_gasoline, on = ['Date'], how = 'inner')
df_temp_2 = pd.merge(df_temp_1, df_gasoline_supply, on = ['Date'], how = 'inner')
df_temp_3 = pd.merge(df_temp_2, df_refinery_capacity, on = ['Date'], how = 'inner')
df_temp_4 = pd.merge(df_temp_3, df_crude_exports, on = ['Date'], how = 'inner')
df_temp_5 = pd.merge(df_temp_4, df_crude_imports, on = ['Date'], how = 'inner')
df_temp_5.head()

,Date,crude_price_per_barrel,Stocks_per_thousand_barrels,No_of_days,Utilization_Percentage,Exp_Thousand_Barrels_Per_Day,Imp_Thousand_Barrels_Per_Day
0,03/25/2022,113.69,18971,27.3,92.1,2988,6259
1,03/18/2022,100.43,18344,27.0,91.1,3844,6486
2,03/11/2022,113.39,17123,27.3,90.4,2936,6395
3,03/4/2022,106.80,19062,28.0,89.3,2422,6319
4,02/25/2022,92.18,17714,28.0,87.7,3796,5767


In [137]:
df_merged = pd.merge(df_temp_5, df_gasoline_price, on = ['Date'], how = 'inner')
df_merged.head()

,Date,crude_price_per_barrel,Stocks_per_thousand_barrels,No_of_days,Utilization_Percentage,Exp_Thousand_Barrels_Per_Day,Imp_Thousand_Barrels_Per_Day,Gas_Price_Date,Gas_price_per_gallon
0,03/25/2022,113.69,18971,27.3,92.1,2988,6259,03/28/2022,4.152
1,03/18/2022,100.43,18344,27.0,91.1,3844,6486,03/21/2022,4.165
2,03/11/2022,113.39,17123,27.3,90.4,2936,6395,03/14/2022,4.252
3,02/25/2022,92.18,17714,28.0,87.7,3796,5767,02/28/2022,3.554
4,02/18/2022,92.89,17624,28.5,87.4,2686,6828,02/21/2022,3.480


In [138]:
df_merged.to_csv("Gas_Price_Data_Merged.csv", index = False)